# Prime Obsession

A notebook to reproduce a few results from "Prime Obsession" by John Derbyshire.

The key equation (from p328) is:
$$
\begin{equation}
\large{J(x) = Li(x) - \sum_{\rho} x^\rho - \log{2} + \int_x^\infty \frac{\mathrm{d}t}{t(t^2 - 1)\log{t}}}
\end{equation}
$$

Start with the imports for the libraries we are using:

In [2]:
from mpmath import *
import pandas as pd
from sympy import mobius
from tqdm.notebook import trange
mp.dps = 30; mp.pretty = True

We use a slightly different definition of the logarithmic integral to `li` defined in `mpmath` (and Mathematica) - see note 128 on p390.

In [6]:
def Li(x, rho=1):
    return ei(rho * log(x))

# Calculating $J(20)$

Pages 333 to 343 go through the calculation of $J(20)$. The secondary term is the part that requires the most work. Page 340 has a table of the logarithmic integral for the first 50 pairs of zeros, reproduced here:

In [10]:
for i in range(1, 51):
    z = zetazero(i)
    a = Li(20, z) + Li(20, z.conjugate())
    nprint(a.real, 6)

-0.210768
0.0215632
-0.0535991
-0.00432174
-0.0868451
-0.037716
-0.0046281
-0.0577894
-0.0400277
-0.0595976
0.0563226
-0.0274298
0.0481966
0.00127986
0.0128283
-0.00472225
0.0361164
0.0317626
0.0222196
-0.037927
-0.0332852
-0.00692417
0.0205354
-0.0312052
0.0280167
0.0188243
0.0228139
-0.0301646
0.0208943
0.0275883
0.00801349
0.0279464
0.0159041
-0.0102871
0.0224912
-0.00106082
0.0130158
-0.0191586
-0.018169
-0.0165671
0.0240114
-0.0223427
-0.0225924
-0.000132221
-0.0180932
0.0221559
-0.017333
-0.0150514
0.0206192
0.0207551


As Derbyshire says on p342 you have to add up thousands of terms to get a few decimal places of accuracy.
Using `zetazero` is slow, so we can use [Andrew Odlyzko's tables of zeros](http://www.dtc.umn.edu/~odlyzko/zeta_tables/index.html) instead.

In [18]:
import gzip

with gzip.open("data/zeros6.gz", "rt") as f:
    lines = f.readlines()
zetazeros = [mpc(0.5, float(s.strip())) for s in lines]
zetazeros[:10]

[(0.5 + 14.1347251420000006305599526968j),
 (0.5 + 21.0220396389999990560681908391j),
 (0.5 + 25.0108575799999997002487361897j),
 (0.5 + 30.4248761260000009087889338844j),
 (0.5 + 32.9350615880000034962904464919j),
 (0.5 + 37.5861781589999992547745932825j),
 (0.5 + 40.9187190119999968374031595886j),
 (0.5 + 43.3270732809999969958880683407j),
 (0.5 + 48.0051508809999987192895787302j),
 (0.5 + 49.77383247800000276583887171j)]

In [28]:
%%time
sec = 0
for i in trange(86000):
    z = zetazeros[i]
    a = Li(20, z) + Li(20, z.conjugate())
    sec += a
nprint(sec.real, 6)

  0%|          | 0/86000 [00:00<?, ?it/s]

-0.370822
CPU times: user 14.5 s, sys: 209 ms, total: 14.7 s
Wall time: 14.6 s


We can calculate the other terms to find the value of $J(20)$:

In [29]:
J20 = Li(20) - sec.real - log(2) + quad(lambda t: 1/(t*(t*t-1)*log(t)), [20, inf])
nprint(J20, 5)

9.5833


This is the same as the result on p343.

# Calculating $\pi(1,000,000)$

Pages 343 and 344 have a calculation for $\pi(1,000,000)$, using the formula:

$$
\begin{equation}
\large{\pi(x) = \sum_{N} \frac{\mu(N)}{N}J(\sqrt[N]{x})}
\end{equation}
$$

This series is finite, since $\sqrt[n]{x} < 2$ for large enough $n$, and $J(x) = 0$ for $x < 2$.

In [30]:
x = 1000000

Define a helper function that yields values of $N$ (and $y = \sqrt[N]{x}$) for which $y$ is at least 2, and the Möbius function is non-zero:

In [32]:
def Ns(x):
    N = 1
    y = x
    while y >= 2:
        if mobius(N) != 0:
            yield N, y
        N += 1
        y = x ** (1/N)
list(Ns(x))

[(1, 1000000),
 (2, 1000.0),
 (3, 99.99999999999997),
 (5, 15.848931924611136),
 (6, 9.999999999999998),
 (7, 7.196856730011519),
 (10, 3.9810717055349727),
 (11, 3.5111917342151315),
 (13, 2.894266124716751),
 (14, 2.6826957952797255),
 (15, 2.51188643150958),
 (17, 2.2539339047347906),
 (19, 2.0691380811147897)]

Now we can reproduce Table 21-1 on p344.

First, let's calculate the secondary terms, since those are the most compute intensive.

In [57]:
%%time
secondary_terms = []
for N, y in Ns(x):
    sec = 0
    for i in trange(len(zetazeros)):
        z = zetazeros[i]
        a = Li(y, z) + Li(y, z.conjugate())
        sec += a
    sec = -mobius(N) * sec.real / N
    nprint(sec, 5)
    secondary_terms.append(sec)

  0%|          | 0/2001052 [00:00<?, ?it/s]

-29.832


  0%|          | 0/2001052 [00:00<?, ?it/s]

0.11036


  0%|          | 0/2001052 [00:00<?, ?it/s]

...

-0.0036218


  0%|          | 0/2001052 [00:00<?, ?it/s]

-0.012723


  0%|          | 0/2001052 [00:00<?, ?it/s]

-0.022414
CPU times: user 1h 13min 57s, sys: 58.4 s, total: 1h 14min 55s
Wall time: 1h 14min 50s


In [58]:
N_values = []
principal_terms = []
log2_terms = []
integral_terms = []
for N, y in Ns(x):
    N_values.append(N)
    principal_terms.append(mobius(N) * Li(y) / N)
    log2_terms.append(-mobius(N) * log(2) / N)
    integral_terms.append(mobius(N) * quad(lambda t: 1/(t*(t*t-1)*log(t)), [y, inf]) / N)

In [77]:
d = {
    "N": N_values,
    "Principal term": principal_terms,
    "Secondary term": secondary_terms,
    "Log 2 term": log2_terms,
    "Integral term": integral_terms
}
df = pd.DataFrame(d).set_index("N")
df["Row totals"] = df["Principal term"] + df["Secondary term"] + df["Log 2 term"] + df["Integral term"]
df = df.astype(float)
df.style.format(precision=5)

,Principal term,Secondary term,Log 2 term,Integral term,Row totals
N,,,,,
1,78627.54916,-29.83162,-0.69315,0.00000,78597.02440
2,-88.80483,0.11036,0.34657,-0.00000,-88.34789
3,-10.04205,0.29990,0.23105,-0.00000,-9.51110
5,-1.69303,0.08785,0.13863,-0.00012,-1.46667
6,1.02760,-0.02349,-0.11552,0.00031,0.88889
7,-0.69393,-0.04737,0.09902,-0.00058,-0.64286
10,0.29539,-0.02791,-0.06931,0.00183,0.20000
11,-0.23615,-0.00634,0.06301,-0.00234,-0.18182
13,-0.15890,0.03206,0.05332,-0.00340,-0.07692


In [78]:
totals = pd.DataFrame(df.sum(), columns=["Column totals"])
totals.style.format(precision=5)

,Column totals
Principal term,78527.34662
Secondary term,-29.46111
Log 2 term,0.03515
Integral term,-0.00799
Row totals,78497.91267


It's interesting that even with over 2 million zeros, the total is only accurate to one decimal place: it's about 0.09 away from the value of $\pi(1,000,000)$ of 78498.

The calculation in the book has a closer agreement, which suggests more than 2 million zeros were used. (The author doesn't say how many were used.)